In [21]:
import pandas as pd 
import re 

In [22]:
url = 'https://drive.google.com/file/d/130KYOX8O4wrP_T8vdz2GfvJRQ03ONmE7/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path, index_col =[0])
df.head(2)

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,...,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,...,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126


In [23]:
df = df[df['Адрес'].str.startswith('Москва')]
df.reset_index(inplace = True)

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,...,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157


In [24]:
# Сохраняем датасет без региональных объявлений
df.to_csv('data_cleaned.csv')

ID

In [ ]:
#переименовали колонку с ID объявления, Тип
df.rename({'ID  объявления': 'ID', 'Тип': 'Type'}, axis='columns')

Количество комнат

In [ ]:
# Обрабатываем колонку с количеством комнат, разделяем на число комнат, планировка. number_of_rooms, rooms_plan  
df[['number_of_rooms','rooms_plan']] = df['Количество комнат'].apply(lambda x: pd.Series(str(x).split(", ")))
# Заполняем нан в  number_of_rooms, rooms_plan
max_room_count = int(df['number_of_rooms'].value_counts(dropna = False).idxmax())
df['number_of_rooms'] = df['number_of_rooms'].map(lambda x: max_room_count if  x == 'nan' else x)
df['rooms_plan'] = df['rooms_plan'].map(lambda x:  x if  x == x else 'Неизвестно')

Площадь квартиры

In [ ]:
# Обрабатываем колонку с площадью квартиры, берём только общую площадь
df['flat_area'] = [row.split('/')[0] for row in df['Площадь, м2']]
#потом  дроп старой колонки 

Дом (этаж,этажность тип)

In [ ]:
#Дом. разбиваем колонку. flat_floor,max_floor,house_type
df['flat_floor'] = [int(row.split('/')[0]) for row in df['Дом']]
df['max_floor'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['house_type'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
#а потом дропнем  старый Дом)))# df = df.drop(['Дом'], axis = 1)


In [ ]:
# дропнуть Площадь комнат

# Лена

# Телефоны

In [ ]:
# Разделяем телефоны на два столбца, чтоб проще проводить фильтрацию
df['phone_1'] = 0
df['phone_2'] = 0
for index in range(len(df)):
    phones = df['Телефоны'][index].split(',')
    df['phone_1'][index] = phones[0]
    if len(phones)==2:
        df['phone_2'][index] = phones[1] 
    else:
        continue

#Балконы

In [ ]:
# Заменяем все значения Nan на 0, т.к. отсутсвие данных скорее всего говорит об отсуствие балкона
df['Балкон'].fillna(0, inplace =True) 

# создаем новые столбцы, куда будме вносить данные по количеству балконов в зависимости от их типа
df['Balcony'] = 0
df['Loggia'] = 0
df['No_balcony'] = 0

# заполняем  столбцы в зависимости
for index in range(len(df)):
    
    if df["Балкон"][index] !=0:
        
        if 'Балкон' in df["Балкон"][index] and 'Лоджия' in df["Балкон"][index]: # проверка на оба типа
            df['Balcony'][index] = df["Балкон"][index].split(', ')[0].split(' ')[1].replace('(', '').replace(')','')
            df['Loggia'][index] = df["Балкон"][index].split(', ')[1].split(' ')[1].replace('(', '').replace(')','')
        
        elif 'Балкон' in df["Балкон"][index]:
            df['Elevator_pass'][index]=df["Балкон"][index].split(' ')[1].replace('(', '').replace(')','')
        
        elif 'Лоджия' in df["Балкон"][index]:
            df['Loggia'][index]=df["Балкон"][index].split(' ')[1].replace('(', '').replace(')','')
    
    else:
        df['No_balcony'][index] = 1

In [ ]:
# дропнуть "Балкон"

# Высота потолков

In [ ]:
false_ceil = df['Высота потолков, м'][df['Высота потолков, м'] > 5]
false_ceil
# из всех "ошибочных" значений, все подчиняются закономерности (трехзначные цифры в мм, двухзначные в см, 
# остальное просто высокие потолки *звуки зависти*), кроме одного значения с индексом 6016

In [ ]:
df.loc[6016]
# изучив объявление (по ссылке), делаем вывод, что скорее всего опечатка
df.loc[6016]['Высота потолков, м'] = 2.8
df['Ceiling_height'] = 0
for index in range(len(df)):
    try:
        if 20<=round(df["Высота потолков, м"][index])<=99:
            df['Ceiling_height'][index] = df["Высота потолков, м"][index]/10
        elif round(df["Высота потолков, м"][index])>99:
            df['Ceiling_height'][index] = df["Высота потолков, м"][index]/100
        else:
            df['Ceiling_height'][index] = df["Высота потолков, м"][index]
    except ValueError:
        df['Ceiling_height'][index] = df['Высота потолков, м'].median()

In [ ]:
# дропнуть высоту потолков

# Мусоропровод

In [ ]:
# скорее всего отсутствие информации говорит об отсуствии предмета
df['Мусоропровод'].fillna('Нет')

# Егор

## Чищу Метро

PS раскомментировать мои можно ctrl+k+c

In [ ]:
df['Метро'].fillna('a (b c', inplace = True)
df['Метро'] = df['Метро'].str.replace(')', '').str.split('(')

<ipython-input-10-40a2f54eeecd>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Метро'] = df['Метро'].str.replace(')', '').str.split('(')


In [ ]:
df_metro = pd.DataFrame(df["Метро"].to_list(), columns=['Station', 'Time1'])
df_metro["Time1"] = df_metro["Time1"].str.split(' ', 1)

df_metro_1 = pd.DataFrame(df_metro["Time1"].to_list(), columns=['Time', 'Vehicle'])

In [ ]:
df_metro = pd.concat([df_metro, df_metro_1], axis = 1)
df_metro.drop('Time1', axis = 1, inplace = True)
df_metro['Time'].replace('None', 'b', inplace = True)

## Чищу Цену 

In [ ]:
price = df['Цена']
splt = price.apply(lambda x: x.split(','))
rent = price.apply(lambda x: x.split(',')[0]).apply(lambda x: float(''.join(
    re.findall("\d+\.\d+", x))))

deposit = price.apply(lambda x: x.split(',')[1]).apply(lambda x: ''.join(re.findall("\d+", x))).apply(
                    lambda x: float(x) if x != '' else 0)

In [ ]:
rent.rename('Rent', inplace = True)
deposit.rename('Deposit', inplace = True)
price_df = pd.concat([rent, deposit], axis = 1)

## Чищу Дети/Животные 

In [ ]:
anchil_df = pd.get_dummies(df['Можно с детьми/животными'])

anchil_df.rename(columns = {'Можно с детьми' : 'Children', 'Можно с детьми, Можно с животными' :
                  'Children_Animals', 'Можно с животными' : 'Animals'}, inplace = True)

## Чищу Дополнительно  

In [ ]:
dop_df = df['Дополнительно'].apply(lambda x: x.split(', ') if x == x else x).fillna('d')
dop_df = pd.get_dummies(dop_df.apply(pd.Series).stack()).sum(level=0)
dop_df.rename(columns = ({'d' : 'NaN', 'Ванна' : 'Bathtab', 'Душевая кабина' : 'Shower Cabin',
                         'Интернет' : 'Internet', 'Кондиционер' : 'Air Conditioner', 'Мебель в комнатах' : 
                         'Furniture Rooms', 'Мебель на кухне' : 'Furniture Kitchen',
                         'Посудомоечная машина' : 'Dishwasher', 'Стиральная машина' : 'Washing Machine',
                         'Телевизор' : 'TV', 'Телефон' : 'Telephone', 'Холодильник' : 'Fridge'}),
              inplace = True)

<ipython-input-16-abedec94e8a2>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dop_df = pd.get_dummies(dop_df.apply(pd.Series).stack()).sum(level=0)


## Чищу Санузел))


In [ ]:
toil = df['Санузел']
toil_df = toil.str.split(',')
toil_df.fillna('e', inplace = True)
toil_df = pd.get_dummies(toil_df.apply(pd.Series).stack()).sum(level=0)

<ipython-input-17-ce737d261657>:4: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  toil_df = pd.get_dummies(toil_df.apply(pd.Series).stack()).sum(level=0)


In [ ]:
al = toil_df.filter(like='Раздельный (1)')
be = toil_df.filter(like='Раздельный (2)')
ga = toil_df.filter(like='Раздельный (3)')
de = toil_df.filter(like='Раздельный (4)')

sep_1 = pd.Series(al.iloc[:, 0] | al.iloc[:, 1], name = 'Sep_Toil_1')
sep_2 = pd.Series(be.iloc[:, 0] | be.iloc[:, 1], name = 'Sep_Toil_2')
sep_3 = pd.Series(ga.iloc[:, 0] | ga.iloc[:, 1], name = 'Sep_Toil_3')
sep_4 = pd.Series(de.iloc[:, 0] | de.iloc[:, 1], name = 'Sep_Toil_4')

In [ ]:
answer = pd.concat([sep_1, sep_2, sep_3, sep_4, toil_df['e'], toil_df.loc[:, 'Совмещенный (1)':]], axis = 1)
answer.rename(columns = ({'Совмещенный (1)' : 'Comb_Toil_1', 'Совмещенный (2)' : 'Comb_Toil_2',
                          'Совмещенный (3)' : 'Comb_Toil_3', 'Совмещенный (4)' : 'Comb_Toil_4',
                         'e' : 'No Description'}), inplace = True)